## DECISION TREE IMPLEMENTATION FROM SCRATCH
### IRIS DATASET



In [1]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
import numpy as np
import math
iris = datasets.load_iris()
X = iris.data
y = iris.target
#to split the dataset into test and train
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=0)


In [2]:
# decision tree class
class Decision_Tree(object):
    
    def __init__(self, max_depth, min_size):
# maximum depth of tree and minimum number of split as input to the class
        self.max_depth = max_depth
        self.min_size = min_size
        self.labels=[]
        


    def split_info(self,groups):
        total=0
        for group in groups:
            total=total+len(group)
            
        split_info=0.0
        for group in groups:
            probability=float(len(group)/total)
            if probability!=0.0:
                split_info-= probability*math.log(probability)
            
        return split_info
 


 # calculates entropy of group
    def entropy(self,groups):
        entries = 0
        labels = {}
        for group in groups:
            entries=entries+len(group)
            for row in group:
                label = row[-1]
                if label not in labels.keys():
                    labels[label] = 0
                labels[label] += 1
                
                
        entropy = 0.0
        for key in labels:
            probability = float(labels[key])/entries
            if probability!=0.0:
                entropy -= probability * math.log(probability)
        return float(entropy)  
        
        
# calculates entropy of singles     
    def entropy_single(self,group):
        labels = {}
        entries=len(group)
        for row in group:
                label = row[-1]
                if label not in labels.keys():
                    labels[label] = 0
                labels[label] += 1
                
        entropy = 0.0       
        for key in labels:
            probability = float(labels[key])/entries
            if probability!=0.0:
                entropy -= probability * math.log(probability)
        return float(entropy)  
        
        
        
#claculates informatio gain     
    def info_gain(self,groups):
        total=0
        for group in groups:
            total=total+len(group)
        info_gain = self.entropy(groups)
        
        for group in groups:
#             entropy= self.entropy(group)
            labels = {}
            for row in group:
                label = row[-1]
                if label not in labels.keys():
                    labels[label] = 0
                labels[label] += 1
            entropy=0.0    
            for key in labels:
                probability=float(labels[key]/len(group))
                if probability!=0.0:
                    entopy=entropy-probability*math.log(probability)
            info_gain=info_gain-float(entropy*(float)(len(group)/total))
                                         
        return info_gain
                
        
        
        
        
        
#calculates gain ratio
    def gain_ratio(self,groups):
        split_info=self.split_info(groups)
        info_gain=self.info_gain(groups)
        if split_info==0:
            return 0.0
        gain_ratio=float(info_gain/split_info)
        return gain_ratio
        

    
 # Split a dataset based on an feature and its value
    def test_split(self,index, value, xy_train):
        left, right = list(), list()
        for row in xy_train:
            if row[index] <= value:
                left.append(row)
            else:
                right.append(row)
        return left, right
    
    
    
    
# Select the best split point for a dataset
    def get_split(self,xy_train):
    #class_values has all distinct labels value
        class_values = [set(row[-1] for row in xy_train)]

        best_index, best_value, best_score, best_groups = -1, -1, -1, None
        #call test_split for every value of every feature 
        #and find the best split with maximum information gain
        for index in range(len(xy_train[0])-1):
            for row in xy_train:
                groups = self.test_split(index, row[index], xy_train)
                info_gain = self.info_gain(groups)
                if info_gain > best_score:
                    best_index, best_value, best_score, best_groups = index, row[index], info_gain, groups

        return {'index':best_index, 'value':best_value, 'groups':best_groups}

    
# Create a terminal node value
    def to_terminal(self,group):
        outcomes = [row[-1] for row in group]
        return max(set(outcomes), key=outcomes.count)
    
    
    
    
# Create child splits for a node or make terminal
    def split(self,node,depth):
        left, right = node['groups']
      
        # check for a no split
        
            
        if not left or not right:
            node['left'] = node['right'] = self.to_terminal(left + right)
            return
        # check for max depth
        
        if depth >= self.max_depth:
            node['left'], node['right'] = self.to_terminal(left), self.to_terminal(right)
            return
        
        # process left child
        if self.entropy_single(left)==0.0:
            node['left'] = self.to_terminal(left)
        
        elif len(left) <= self.min_size:
            node['left'] = self.to_terminal(left)
        else:
            node['left'] = self.get_split(left)
            self.split(node['left'], depth+1)
        # process right child
        if self.entropy_single(right)==0.0:
            node['right'] = self.to_terminal(right)
        elif len(right) <= self.min_size:
            node['right'] = self.to_terminal(right)
        else:
            node['right'] = self.get_split(right)
            self.split(node['right'],depth+1)
            
            
            
# to build a decision tree
    def build_tree(self):
        self.root = self.get_split(self.xy_train)
        self.split(self.root, 1)
        return self.root
    #root is the root node of tree
    
     
    
    
#     Print a decision tree
    def print_tree(self, node, depth):
        
        print("LEVEL ",depth)
        if isinstance(node, dict):
     
            
            
            lblctr={}
            
            for group in node['groups']:
                for row in group:
                    label=row[-1]
                    if label not in lblctr:
                        lblctr[label] = 0
                    lblctr[label] += 1
                
            for key,value in lblctr.items():
                print("Count of ",key ," = ",value)
#             print(node['groups'])
            print("entropy is ",self.entropy(node['groups']))
#             print("split info is ",self.split_info(node['groups']))
           
            print("Splitting on feature ", node['index']+1)
            print("with gain ratio ",self.gain_ratio(node['groups']))
            print()
            
            self.print_tree(node['left'], depth+1)
            self.print_tree(node['right'], depth+1)
            
        else:
            
            print("Reached Leaf node " )
            print("predicted value at this node is ",node)
            print()
            
            

   

 #  function to call print_tree
    def display_tree(self):
        print("The Tree formed is as follows:\n")
        self.print_tree(self.root,0)
       


            
    
# Make a prediction with a decision tree
    def predict(self,node, row):
        if row[node['index']] < node['value']:
            if isinstance(node['left'], dict):
                return self.predict(node['left'], row)
            else:
                return node['left']
        else:
            if isinstance(node['right'], dict):
                return self.predict(node['right'], row)
            else:
                return node['right']
            
            
# function to call predict in each row of test data           
    def predict_tree(self,x_test):
        self.y_pred = np.array([])
        for i in x_test:
            self.y_pred = np.append(self.y_pred,self.predict(self.root,i))
            
        return self.y_pred
        
        
        
# fit function will merge x_train and y_train and call build_tree          
    def fit(self,X,y):
        self.X = X
        self.y = y
        self.xy_train = np.column_stack((X, y))
        self.labels=[set(row[-1] for row in self.xy_train)]
#         print(self.labels)
        self.build_tree()

            

In [3]:
dt = Decision_Tree(max_depth=3,min_size=2)

In [4]:
dt.fit(x_train,y_train)

In [5]:
dt.display_tree()

The Tree formed is as follows:

LEVEL  0
Count of  1.0  =  34
Count of  0.0  =  37
Count of  2.0  =  41
entropy is  1.0956714129052516
Splitting on feature  1
with gain ratio  1.5987869524252456

LEVEL  1
Count of  1.0  =  21
Count of  0.0  =  37
Count of  2.0  =  5
entropy is  0.879862924451871
Splitting on feature  1
with gain ratio  0.0

LEVEL  2
Reached Leaf node 
predicted value at this node is  0.0

LEVEL  2
Reached Leaf node 
predicted value at this node is  0.0

LEVEL  1
Count of  2.0  =  36
Count of  1.0  =  13
entropy is  0.5785341056326687
Splitting on feature  1
with gain ratio  0.9158449197401396

LEVEL  2
Count of  2.0  =  22
Count of  1.0  =  11
entropy is  0.6365141682948128
Splitting on feature  1
with gain ratio  0.0

LEVEL  3
Reached Leaf node 
predicted value at this node is  2.0

LEVEL  3
Reached Leaf node 
predicted value at this node is  2.0

LEVEL  2
Count of  2.0  =  14
Count of  1.0  =  2
entropy is  0.37677016125643675
Splitting on feature  1
with gain ratio 

In [6]:
y_test_pred=dt.predict_tree(x_test)

In [7]:
#predicted values
y_test_pred

array([0., 2., 0., 2., 0., 2., 0., 2., 2., 2., 2., 2., 2., 2., 2., 0., 2.,
       0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 2., 0., 0., 2., 0., 0., 2.,
       2., 0., 0., 2.])

In [8]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_test_pred)


0.5263157894736842